# Syntax and the Corresponding Magic

```
with x as y:
    "body"
```

---

```
y = x.__enter__()
try:
    "body"
except BaseException as exc:
    x.__exit__(*sys.exc_info())  # type(exc), exc, traceback
```

---
---

```
for y in x:
    print(f"I like {y}")
```

---

```
try:
    ix = x.__iter__()
    while True:
        y = next(ix)
        print(f"I like {y}")
except StopIteration:
    pass
```

In [437]:
class Mgr:
    def __init__(self, name):
        self.name = name
        
    def __enter__(self):
        print('enter')
        
    def __exit__(self, exc_type, exc_info, tb):
        print('exit!')

In [438]:
with Mgr('nick'):
    aaaaaa

enter
exit!


NameError: name 'aaaaaa' is not defined

In [344]:
class Base:
    def __init__(self, name):
        self.name = name

In [372]:
b = Base('NaOH')

In [373]:
b

In [374]:
print(repr(b))

```
















```

## Stringy Types

In [375]:
class Stringy:
    def __init__(self, name):
        self.name = name
        
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.name!r})"
    
    def __str__(self) -> str:
        return f"It's name...is {self.name}"

In [376]:
a = Stringy('apple')
a

Stringy('apple')

In [377]:
repr(a)

"Stringy('apple')"

In [378]:
str(a)

"It's name...is apple"

In [74]:
class StringyToo:
    def __init__(self, name):
        self.name = name
        
    def to_string(self):
        return f"It's name...is {self.name}"

In [77]:
b = StringyToo('cheese')
b

In [79]:
b.to_string()

"It's name...is cheese"

```
















```

### Quacks like...

String-like objects can be used in place of other strings

In [321]:
f"I have an object. {b}."

'I have an object. <__main__.Base object at 0x7fbbc22e49b0>.'

In [90]:
f"I have an object. {a}. It was made with {a!r}"

"I have an object. It's name...is apple. It was made with Stringy('apple')"

```
















```

## Addable Types

Implementing the mathematical magic methods can let you use any of the infix operators to manipulate your objects as makes sense.

    x + y      x ** y      x & y      x << y
    x - y      x // y      x | y      x >> y
    x * y      x % y       x ^ y
    x / y      x @ y       

In [406]:
class Mathy(Stringy):
    def __add__(self, other) -> str:
        return f"{self.name} and {other.name}"

In [166]:
Mathy("coke") + Mathy("mentos")

'coke and mentos'

In [167]:
Mathy("eggs") + Mathy("spam") + Mathy("cheese")

TypeError: must be str, not Mathy

In [199]:
class Addy(Stringy):
    def __add__(self, other) -> 'Addy':
        return Addy(f"{self.name} and {other.name}")

In [173]:
Addy("eggs") + Addy("spam") + Addy("cheese")

Addy('eggs and spam and cheese')

```
















```

### Builtins that Add

The built-in `sum()` uses `+` to add objects together. It uses a start value to 

In [407]:
help(sum)

Help on built-in function sum in module builtins:

sum(iterable, start=0, /)
    Return the sum of a 'start' value (default: 0) plus an iterable of numbers
    
    When the iterable is empty, return the start value.
    This function is intended specifically for use with numeric values and may
    reject non-numeric types.



In [194]:
sum([Addy("eggs"), Addy("spam"), Addy("cheese"), Addy('funny walks')])

TypeError: unsupported operand type(s) for +: 'int' and 'Addy'

In [196]:
class Addy2(Stringy):
    def __add__(self, other) -> 'Addy2':
        return Addy2(f"{self.name} and {other.name}")
    
    def __radd__(self, other) -> 'Addy2':
        if not other:
            return self

In [197]:
sum([Addy2("eggs"), Addy2("spam"), Addy2("cheese"), Addy2('funny walks')])

Addy2('eggs and spam and cheese and funny walks')

```
















```

## Orderable Types

In [384]:
doctors = [
    Stringy('Eccleston'), 
    Stringy('Tennant'), 
    Stringy('Smith'), 
    Stringy('Capaldi'), 
    Stringy('Whittaker'),
]

sorted(doctors, key=lambda i: i.name)

[Stringy('Capaldi'),
 Stringy('Eccleston'),
 Stringy('Smith'),
 Stringy('Tennant'),
 Stringy('Whittaker')]

```
















```

In [385]:
class Comparable(Stringy):
    def __lt__(self, other):
        return self.name < other.name

In [386]:
Comparable('apple') < Comparable('banana')

True

In [276]:
Comparable('pear') < Comparable('banana')

False

In [387]:
doctors = [Comparable(d.name) for d in doctors]
doctors

[Comparable('Eccleston'),
 Comparable('Tennant'),
 Comparable('Smith'),
 Comparable('Capaldi'),
 Comparable('Whittaker')]

In [388]:
sorted(doctors)

[Comparable('Capaldi'),
 Comparable('Eccleston'),
 Comparable('Smith'),
 Comparable('Tennant'),
 Comparable('Whittaker')]

```
















```

## Container Types

In [401]:
import re

import requests


def get_xkcd_hotlink(comic_number):
    response = requests.get(f"https://xkcd.com/{comic_number}/")
    match = re.search(
        r"Image URL \(for hotlinking/embedding\):\s*(?P<url>http.*(jpg|png))", 
        response.text,
    )
    if not match:
        return None

    src = match.groupdict()["url"]
    return src        


class XKCDContainer:        
    def __getitem__(self, key):
        url = get_xkcd_hotlink(key)
        if url is None:
            raise KeyError("couldn't find XKCD comic with that number")
        return url
        
    def __contains__(self, key):
        return get_xkcd_hotlink(key) is not None

In [402]:
xkcd_container = XKCDContainer()

In [403]:
xkcd_container[364]

'https://imgs.xkcd.com/comics/responsible_behavior.png'

In [404]:
1000 in xkcd_container

True

In [405]:
10000 in xkcd_container

False

```
















```

## Proprietary Magic

In [398]:
class XKCDComic:
    def __init__(self, url):
        self.url = url

    def _repr_html_(self):
        return f"<img src='{self.url}'>"


class XKCDViewer(XKCDContainer):
    def __getitem__(self, key):
        return XKCDComic(super().__getitem__(key))

In [399]:
xkcd = XKCDViewer()

In [400]:
xkcd[843]